In [48]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup as bs 
import requests 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [49]:
url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [50]:
res_hoteles = requests.get(url_hoteles) 
res_hoteles.status_code

200

In [51]:
sopa_hoteles = bs(res_hoteles.content, "html.parser")
sopa_hoteles

<!DOCTYPE html>

<html data-n-head-ssr="">
<head><script data-dtconfig="rid=RID_254173350|rpid=1091923639|domain=accor.com|reportUrl=/rb_80be963f-a859-4808-9b2a-ceb8d44df738|app=1a152145cd696e21|featureHash=ICA7NVfhqrux|srsr=50000|xb=https:^bs/^bs/www^bs.google-analytics^bs.com^bs/collect^bs?.*^p .*/cdx/platform.html|rdnt=1|uxrgce=1|cuc=85aac0bj|mdl=mdcc5=20|mel=100000|expw=1|dpvc=1|md=mdcc2=cJSESSIONID,mdcc3=corg,mdcc4=bsessionStorage.userData,mdcc5=bCS_CONF.integrations_handler.replaylink,mdcc6=bCS_CONF.integrations_handler.sessionID,mdcc7=cplatform,mdcc8=fx-cdn-forward|lastModification=1738156612732|mdp=mdcc3|tp=500,50,0|agentUri=/ruxitagentjs_ICA7NVfhqrux_10285240307101407.js" src="/ruxitagentjs_ICA7NVfhqrux_10285240307101407.js" type="text/javascript"></script><script type="text/javascript">var kameleoonLoadingTimeout = 1000;window.kameleoonQueue = window.kameleoonQueue || [];window.kameleoonStartLoadTime = new Date().getTime();if (!document.getElementById('kameleoonLoadingStyleSh

In [52]:
hoteles = sopa_hoteles.find_all("div", {"class": "hotelblock__content"})[:10]
hoteles

[<div class="hotelblock__content" data-v-40d92af9=""><header class="title-block" data-v-40d92af9="" data-v-438ba70d=""><h2 class="title" data-v-438ba70d="" data-v-71eed73c=""><a class="title__link title__link" data-v-71eed73c="" href="/ssr/app/ibis/rates/8052/index.es.shtml?compositions=1&amp;dateIn=2025-02-16&amp;nights=1&amp;hideHotelDetails=false&amp;hideWDR=false&amp;destination=madrid-spain" target="" title="ibis Styles Madrid Prado">
     ibis Styles Madrid Prado
     <span class="title-block__stick-to-previous-text" data-v-438ba70d=""><span class="rating" data-v-399a1766="" data-v-438ba70d=""><span class="sr-only" data-v-399a1766="">3 Estrellas</span> <svg aria-hidden="true" class="icon rating__star" data-v-399a1766="" focusable="false" viewbox="0 0 7 7" xmlns="http://www.w3.org/2000/svg"><path d="M3.5 5.52 1.34 6.66l.41-2.41L0 2.55l2.42-.36L3.5 0l1.08 2.2L7 2.53l-1.75 1.7.41 2.42z"></path></svg><svg aria-hidden="true" class="icon rating__star" data-v-399a1766="" focusable="fals

In [53]:
for hotel in hoteles:
    titulo = hotel.find("a", {"class": "title__link title__link"}).get_text()
titulo.split("   ")[2].split(" ")[1]

'4'

NO SE POR QUE NO ME DEVUELVE LOS 10 PRIMEROS HOTELES QUE YO VEO EN LA PAGINA WEB 

NO SE COMO SCRAPEAR EL PRECIO (NO ME SALE EN EL HTML)

In [54]:
def scrap_info_hoteles(url):
    dictio_scrap = {"nombre_hotel": [],
                    "estrellas": []}
    res_hotel = requests.get(url)
    if res_hotel.status_code == 200:
        sopa_hotel = bs(res_hotel.text, "html.parser")
        info_hoteles = sopa_hotel.find_all("div", {"class": "hotelblock__content"})[:10]
        for hotel in info_hoteles: 
            titulo = hotel.find("a", {"class": "title__link title__link"}).get_text().split("   ")[1].strip()
            estrellas = hotel.find("a", {"class": "title__link title__link"}).get_text().split("   ")[2].split(" ")[1]
            dictio_scrap["nombre_hotel"].append(titulo)
            dictio_scrap["estrellas"].append(estrellas)
    return dictio_scrap

In [55]:
dictio_final = scrap_info_hoteles(url_hoteles)
df_hoteles_competencia = pd.DataFrame(dictio_final)
df_hoteles_competencia

,nombre_hotel,estrellas
0,ibis Styles Madrid Prado,3
1,Mercure Madrid Centro,4
2,ibis budget Madrid Centro Lavapies,1
3,ibis Styles Madrid Centro Maravillas,2
4,Novotel Madrid Center,4
5,ibis budget Madrid Calle 30,1
6,ibis Madrid Centro las Ventas,2
7,ibis budget Madrid Centro las Ventas,1
8,Novotel Madrid City Las Ventas,4
9,ibis Styles Madrid City Las Ventas,4


In [56]:
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url_hoteles)

precios_hoteles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper"))
        )

for precio_hotel in precios_hoteles[:10]:  
    precio_final = precio_hotel.text.split('\n')[1].replace("€", "")
print(precio_final)

driver.quit()

100


In [57]:
def scrap_info_hoteles(url):
    dictio_scrap = {"nombre_hotel": [],
                    "estrellas": [],
                    "precio_noche": []}
    res_hotel = requests.get(url)
    if res_hotel.status_code == 200:
        sopa_hotel = bs(res_hotel.text, "html.parser")
        info_hoteles = sopa_hotel.find_all("div", {"class": "hotelblock__content"})[:10]
        for hotel in info_hoteles: 
            dictio_scrap["nombre_hotel"].append(hotel.find("a", {"class": "title__link title__link"}).get_text().split("   ")[1].strip())
            dictio_scrap["estrellas"].append(hotel.find("a", {"class": "title__link title__link"}).get_text().split("   ")[2].split(" ")[1])
    
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url_hoteles)

    precios_hoteles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper"))
        )

    for precio_hotel in precios_hoteles[:10]: 
        dictio_scrap["precio_noche"].append(precio_hotel.text.split('\n')[1].replace("€", ""))
    
    driver.quit()

    return dictio_scrap


In [58]:
dictio_final = scrap_info_hoteles(url_hoteles)
df_hoteles_competencia = pd.DataFrame(dictio_final)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado,3,179
1,Mercure Madrid Centro,4,188
2,ibis budget Madrid Centro Lavapies,1,106
3,ibis Styles Madrid Centro Maravillas,2,172
4,Novotel Madrid Center,4,95
5,ibis budget Madrid Calle 30,1,116
6,ibis Madrid Centro las Ventas,2,90
7,ibis budget Madrid Centro las Ventas,1,102
8,Novotel Madrid City Las Ventas,4,81
9,ibis Styles Madrid City Las Ventas,4,100
